In [1]:
# default_exp stats_utils

# Statistics Utils

In [2]:
# export
import numpy as np
import pandas as pd
from scipy.stats import boxcox, pearsonr
from scipy.special import inv_boxcox
from pandas.tseries.frequencies import to_offset
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tools.eval_measures import aic, bic
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [3]:
# export
class cPreProcessing():
    """
    Parent class.
    
    Methods for dealing with irregularly spaced or missing data.
    """
    
    def __init__(self):
        pass
    
    def fget_regular_times(self, df, timestep):
        """
        Generate dataframe of regularly spaced times (to impute to)
        (From fbprophet/forecaster/make_future_dataframe)
        
        Parameters
        ----------
        df       = [pandas dataframe]
        timestep = [datetime timedelta object]
        
        
        Returns
        -------
        regtimes = [pandas DatetimeIndex] of datetimes regularly spaced at timestep
        
        """
        # normalize start date to midnight
        start_date = df.ds.min().normalize()

        # round up end date by one extra timestep
        end_date = (df.ds.max() + timestep).normalize()

        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
        regtimes = pd.date_range(start=start_date, end=end_date, freq=to_offset(timestep))

        return regtimes
    
    def finterleaf(self, df, impute_times):
        """
        Interleaf dataframe with new prediction times
        Set values at prediction dates as NaN so can use imputer

        Parameters
        ----------
        df           = [pandas dataframe]
        impute_times = [pandas DatetimeIndex] (format of regtimes)

        Returns
        -------
        dft = pandas dataframe (format for use in fimpute)
        """
        # if impute_times taken from fmake_regular_times()
        if type(impute_times) == pd.core.indexes.datetimes.DatetimeIndex:
            impute_times = pd.DataFrame(impute_times)
            impute_times.columns = ["ds"]

        # set date index
        df.set_index('ds', inplace=True)
        impute_times.set_index('ds', inplace=True)

        # combine (interleaf)
        dft = pd.concat([df, impute_times], sort=True)
        dft.sort_values(by=["ds"], inplace=True)
        
        # remove duplicate entries
        dft = dft[dft.index.duplicated() == False]

        return dft
    
    def fimpute(self, df, method="time"):
        """
        Imputation of data to new timestamps with NaN value.

        Parameters
        ----------
        df = dataframe containing original data and NaNs at timestamps for imputation
             timestamps are the df index
        
        Returns
        -------
        dfi = imputed dataframe
        """
        # interpolates halfway, doesn´t account for weighting towards closer time
        if method == "interp":
            dfi = df.interpolate()

        # missing values given mean value over whole time series
        if method == "mean":
            imp = SimpleImputer(missing_values=np.nan, strategy='mean')
            imp.fit(df)
            dfi = imp.transform(df)

        # linear interpolation weighted by timestamp proximity
        if method == "time":
            dfi = df.interpolate(method='time') 

        # smoothing
        if method == "akima":
            dfi = df.interpolate(method='akima')

        return dfi
    
    def fmake_regular_freq(self, df, timestep, method="time"):
        """
        Interpolate data so regular update frequency throughout dataset.
        (Deal with missing values)
        
        Some python functions (e.g. seasonal_decompose, AutoArima) require a data "freq" argument
        to determine seasonality. (Can be inferred from df.index.freq, df.index.inferred_freq)
        Such functions require a constant data frequency.
        
        Parameters
        ----------
        df       = irregularly space dataframe (with datestamp name "ds")
        timestep = desired update frequency of data (timedelta object)
        method   = imputation method
        
        Returns
        -------
        dff = imputed regularly spaced [pandas dataframe]
        """
        # 0. preprocess: if dataframe alreay has time as index: reset and add as column
        if df.index.name == "ds":
            # df_lim["ds"] = df_lim.index
            df.reset_index(level=0, inplace=True)
        
        # 1. include in dataset times where you want to impute (and set to NaN values)
        impute_times = self.fget_regular_times(df, timestep)
        dft = self.finterleaf(df, impute_times)
        
        # 2. impute with chosen method
        dfi = self.fimpute(dft, method=method)
        
        # 3. remove original data not at correct timestep
        dff = dfi[dfi.index.isin(impute_times) == True]
        
        if dff.index.freq == None:
            dff.index.freq = to_offset(timestep)
        
        return dff

In [4]:
# export
class cStationary(cPreProcessing):
    """
    methods for checking whether time series is stationary
    methods for transforming the time series into a stationary time series
    methods for obtaining (p,q,d) ARIMA parameters
    
    https://towardsdatascience.com/detecting-stationarity-in-time-series-data-d29e0a21e638
    """
    
    def __init__(self):
        pass
    
    def fadf_verbose(self, adftest):
        """
        CHECK STATIONARITY.
        Print explanation of output of Augmented Dickey-Fuller test.
        
        The Augmented Dickey-Fuller test is a type of statistical test called a unit root test. 
        The intuition behind a unit root test is that it determines how strongly a time series is defined by a trend.
        
        Parameters
        ----------
        adftest = adfuller(data.y, regression="ct")
        
        Returns
        -------
        """
        print("""
        Augmented Dickey-Fuller:
        Null hypothesis: the time series can be represented by a unit root, thus not stationary (has some time-dependent structure)
        """)
   
        output = pd.Series(adftest[0:4], index=['Test Statistic','pvalue','#Lags Used','Number of Observations Used'])
        for key,value in adftest[4].items():
            output['Critical Value ({})'.format(key)] = value

        print(output)
 
        if output.pvalue <= 0.05:
            print("\nReject the null hypothesis (H0), the data does not have a unit root and IS STATIONARY.")
            return True
        else:
            print("\nFail to reject the null hypothesis (H0), the data has a unit root and is NON-STATIONARY.")
            return False

    def fkpss_verbose(self, kpsstest):
        """
        CHECK STATIONARITY.
        Print explanation of output of Kwiatkowski-Phillips-Schmidt-Shin test.
        
        Another test for checking the stationarity of a time series (reversed null hypothesis to ADF).
        In KPSS test, to turn ON the stationarity testing around a trend, you need to explicitly pass the regression='ct'.
        A major difference between KPSS and ADF tests:
        the capability of the KPSS test to check for stationarity in the ‘presence of a deterministic trend’.
        
        Parameters
        ----------
        kpsstest = kpss(data.y, regression="ct")
        
        Returns
        -------
        """
        print("""    
        Kwiatkowski-Phillips-Schmidt-Shin:
        Null hypothesis: the process is trend stationary
        """)
    
        output = pd.Series(kpsstest[0:3], index=['Test Statistic','pvalue','Lags Used'])
        for key,value in kpsstest[3].items():
            output['Critical Value ({})'.format(key)] = value
        print (output)

        if output.pvalue <= 0.05:
            print("\nReject the null hypothesis (H0), the data has a unit root and is NON-STATIONARY.")
            return False
        else:
            print("\nFail to reject the null hypothesis (H0),the data does not have a unit root and IS STATIONARY. ")    
            return True
        
    def fstationary_verbose(self, stat_adf, stat_kpss):
        """
        CHECK STATIONARITY.
        Compare results of adf and kpss tests and advise how to make stationary.
        
        Returns
        -------
        """
        if (stat_adf is False) and (stat_kpss is False):
            print("\nBoth tests conclude that the series is not stationary -> series is not stationary")
            return False
        elif (stat_adf is True) and (stat_kpss is True):
            print("\nBoth tests conclude that the series is stationary -> series is stationary")
            return True
        elif (stat_adf is False) and (stat_kpss is True):
            print("\nKPSS = stationary and ADF = not stationary -> trend stationary, use power tranform to make stationary")
            return False
        elif (stat_adf is True) and (stat_kpss is False):
            print("\nKPSS = not stationary and ADF = stationary -> difference stationary, use differencing transform to make stationary")
            return False

        
    def fcheck_stationary(self, y, verbose=True):
        """
        CHECK STATIONARITY.
        
        Parameters
        ----------
        y = time series variable, data.y
        
        Returns
        -------
        stationary status [bool]
        """
        #df.dropna()
        
        adftest  = adfuller(y, regression="ct")
        kpsstest = kpss(y, regression="ct")
        
        if verbose:
            stat_adf  = self.fadf_verbose(adftest)
            stat_kpss = self.fkpss_verbose(kpsstest)
            stat = self.fstationary_verbose(stat_adf, stat_kpss)

        return stat
    
    def fdecompose(self, df, model="additive"):
        """
        CHECK STATIONARITY.
        
        Seasonal decomposition using moving averages
        https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html
        Time series must be regularly spaced (have constant frequency, dff.index.freq or dff.index.inferred_freq)
        
        Parameters
        ----------
        df = data frame with date index (to infer frequency)
        """
        s = seasonal_decompose(df, model=model)
        
        trend = s.trend
        plt.plot(trend)
        plt.title("Trend")
        plt.show()

        seasonal = s.seasonal
        plt.plot(seasonal)
        plt.title("Seasonal component")
        plt.show()

        resid = s.resid
        plt.plot(resid)
        plt.title("Residuals")
        plt.show()
        
    def fcheck_density(self, y):
        """
        CHECK STATIONARITY.
        Plot histogram and density trend (check gaussianity)
        """
        plt.figure(1)
        plt.subplot(211)
        plt.hist(y)
        plt.title("Data Distribution")
        plt.subplot(212)
        y.plot(kind='kde')
        plt.show()
    
    def fcheck_lag(self, y):
        """
        CHECK STATIONARITY.
        Plot lag scatter, autocorrelation and partial autocorrelation functions
        For differencing and establishing (p,q,d) values for ARIMA
        """
        plt.figure()
        pd.plotting.lag_plot(y)
        plt.title("Lag-1 plot")
        plt.plot()

        plt.figure()
        pd.plotting.autocorrelation_plot(y)
        plt.title("Autocorrelation")
        plt.plot()

        # contains confidence interval: 
        # correlation values outside of this code are very likely a correlation and not a statistical fluke
        plot_acf(y)
        plot_pacf(y)
    
    def fdifferencing(self, df, interval=1):
        """
        MAKE STATIONARY. (difference stationary)
        
        adf and kpss can give the d value required by ARIMA
        
        Make series stationary: In order to satisfy the assumption, it is necessary to make the series stationary. 
        This would include checking the stationarity of the series and performing required transformations
        Determine d value: For making the series stationary, the number of times the difference operation was 
        performed will be taken as the d value.
        
        The auro_arima function works by conducting differencing tests 
        (i.e., Kwiatkowski–Phillips–Schmidt–Shin, Augmented Dickey-Fuller or Phillips–Perron) 
        to determine the order of differencing, d. Canova-Hansen test for seasonal stability.
        
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html
        """
        # df.dropna()
        
        return df.diff(periods=interval)

    def fboxcox(self, df):
        """
        MAKE STATIONARY. (trend stationary)
        
        [https://mode.com/example-gallery/forecasting_prophet_python_cookbook/]
        Often in forecasting, you’ll explicitly choose a specific type of power transform to apply to the data 
        to remove noise before feeding the data into a forecasting model (e.g. a log transform or square root 
        transform, amongst others). However, it can sometimes be difficult to determine which type of power 
        transform is appropriate for your data. This is where the Box-Cox Transform comes in. Box-Cox Transforms 
        are data transformations that evaluate a set of lambda coefficients (λ) and selects the value that 
        achieves the best approximation of normality.
        
        Prophet natively models the increase in mean of the data over time, 
        but we should take additional steps to normalize as much variance as possible 
        to achieve the most accurate forecasting results.
        We can do this by applying a power transform to our data.
        
        [https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html]
        
        inverse transform (after fitting and forecasting):
        forecast[['yhat','yhat_upper','yhat_lower']] = forecast[['yhat','yhat_upper','yhat_lower']].apply(lambda x: inv_boxcox(x, lam))


        Parameters
        ----------
        df = dataframe with y component to be transformed
             constraints: data must be positive (and non-zero)
        
        Returns
        -------
        df  = original dataframe with additional tranformed y column
        lam = lambda value of power transform determined by boxcox, needed for inversing the transformation
              constraints: lambda must not be negative
        """
        df['y_trans'], lam = boxcox(df['y'])
        
        return df, lam

In [5]:
# export
class cErrorMetrics():
    """
    methods for evaluating error
    """
    
    def __init__(self):
        pass
    
    def fmse_manual(self, y_true, y_pred):
        """
        Mean square error.
        Manual implementation.
        """
        return np.sum( (np.array(y_true) - np.array(y_pred))**2 ) / len(y_true)
    
    def fmse(self, y_true, y_pred):
        """
        Mean square error.
        (from sklearn.metrics import mean_squared_error)
        """
        return mean_squared_error(y_true, y_pred)
    
    def frms(self, y_true, y_pred):
        """
        Root mean square error.
        (from sklearn.metrics import mean_squared_error)
        """
        return np.sqrt(mean_squared_error(y_true, y_pred))
    
    def fmape(self, y_true, y_pred): 
        """
        mean_absolute_percentage_error.
        (can cause division-by-zero errors)
        """
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    def fcc(self, y_true, y_pred):
        """
        correlation coefficient
        as defined in Gruet et al (2018)
        (cov is a 2x2 matrix?)
        """
        n = np.cov(y_true, y_pred)
        d = np.sqrt(np.var(y_true)*np.var(y_pred))
        return n/d

    def fcc_pearsonr(self, y_true, y_pred):
        """
        pearson correlation coefficient
        """
        return pearsonr(y_true, y_pred)
    
    def faic(self):
        """
        Akaike information criterion.
        An estimator of out-of-sample prediction error and thereby relative quality of statistical models for a given set of data.
        Method of model selection (AutoArima)
        """
        #statsmodels.tools.eval_measures.aic
        pass
    
    def fbic(self):
        """
        Bayes information criterion, Schwarz criterion.
        Criterion for model selection among a finite set of models; the model with the lowest BIC is preferred.
        """
        #statsmodels.tools.eval_measures.bic
        pass

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_stats_utils.ipynb.
Converted 02_plot_utils.ipynb.
Converted 03_read_data.ipynb.
Converted 04_model.ipynb.
Converted 05_solar_flux_time_series_forecasting.ipynb.
Converted 06_ensemble_utils.ipynb.
Converted index.ipynb.
